In [1]:
using Plots
# The GR plotting background is significantly faster
gr();

In [2]:
function v_target(t)
    #return [1, sin(t*2*pi/30)]
    
    # This codes for a zigzag with period 10
    if t % 10 >= 10/2
        return [1, 1]
    else
        return [1, -1]
    end
end

function a_predator(t, k_r, p_r, k_m)
    # Acceleration is directed directly at the prey, and is scaled by the mass
    return normalize(p_r - k_r)/k_m
end

function mass_to_period(kozai_masses)
    # Euler's Method time resolution
    dt = 0.5
    time_max=1000000

    # Preallocate giant arrays and use them repeatedly for speed
    print("Allocating memory...")
    flush(STDOUT)
    kozai = zeros(Int(time_max/dt+1),2)
    y_s = zeros(time_max)
    results = []
    print(" done.\n")
    flush(STDOUT)
    
    for kozai_mass in kozai_masses
        # Set initial conditions
        p_r = [0, 0]
        k_r = [0, 0]
        k_v = [0, 0]
        
        # This is just a fairly general Euler's Method solver for parametric DEs
        for (index, t) in enumerate(0:dt:time_max)
            p_r += v_target(t) .* dt
            k_v += a_predator(t, k_r, p_r, kozai_mass) .* dt

            if norm(k_v) != 0
                k_v = normalize(k_v) .* min(norm(k_v))
            end

            k_r += k_v .* dt

            kozai[index,:] = k_r
        end
        
        # Now we "flatten" the parametric position array [(x, y)...]
        # into a x vs. y array [y1, y2 ...]
        # NOTE: This only makes sense when the parametric curve passes
        # the vertical line test, which, in many cases, it does not.
        fill(y_s, 0)
        for i in 1:size(kozai,1)
            (x, y) = kozai[i,:]
            if ceil(Int, x) > time_max
                # If, as happens by chance, the prey passes max_time,
                # truncate to fit.
                continue
            end
            y_s[ceil(Int, x)] = round(Int,y)
        end
        
        # Make sure the trajectory is centered around zero so the signal is clean
        y_s -= mean(y_s)
        
        # Now convert to frequency space using a Fast Fourier Transform
        # Lots of random constants floating around to make the frequency mappings work.
        dT = 1 # This is 1 because we are working in index space
        yf = 2.0/(time_max/dT) * abs(rfft(y_s))
        xf = linspace(0, 0.5/dT, div(time_max/dT,2))
        
        # A heuristic to extract the "low" frequency modulation. Essentially, 
        # we find the strongest signal, which is the  "high" frequency 
        # oscillation and look for a peak at less than half its frequency.
        period = 1/xf[indmax(yf[10:div(indmax(yf),2)])+9]
        push!(results, period)
        
        @printf("%d\r", kozai_mass)
        flush(STDOUT)
    end
    results
end

function polyfit(x, y, n)
    # Really simple polynomial least-squares fit
    A = [ float(x[i])^p for i = 1:length(x), p = 0:n ]
    A \ y
end

function model(coeff)
    # Produce a model closure from the polyfit output
    function result(x)
        reduce(+, (c*x^(p-1) for (p, c) in enumerate(coeff)))
    end
end;

In [3]:
p_r = [0, 0]
k_r = [0, 0]
k_v = [0, 0]

dt = 0.5

kozai_mass=120
time_max=1000000
kozai = zeros(Int(time_max/dt+1),2)

prey = []

for (index, t) in enumerate(0:dt:time_max)
    p_r += v_target(t) .* dt
    k_v += a_predator(t, k_r, p_r, kozai_mass) .* dt

    if norm(k_v) != 0
        k_v = normalize(k_v) .* min(norm(k_v))
    end

    k_r += k_v .* dt

    kozai[index,:] = k_r
    push!(prey, p_r)
end
@printf("%d frames\n", size(kozai,1))

2000001 frames


In [4]:
plot(kozai[1:min(50000,end),1], kozai[1:min(50000,end),2])

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10000 
 
 
 20000 
 
 
 -50 
 
 
 0 
 
 
 50 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 30.6037,186.727 30.6037,186.736 30.6038,186.749 30.604,186.767 30.6041,186.789 30.6043,186.816 30.6046,186.846 30.6048,186.882 30.6051,186.921 30.6055,186.965 
 30.6058,187.013 30.6063,187.065 30.6067,187.119 30.6072,187.176 30.6077,187.234 30.6083,187.295 30.609,187.356 30.6096,187.417 30.6104,187.479 30.6111,187.541 
 30.6119,187.602 30.6128,187.664 30.6137,187.727 30.6147,187.791 30.6157,187.856 30.6167,187.922 30.6178,187.989 30.619,188.059 30.6202,188.129 30.6214,188.202 
 30.6227,188.277 30.624,188.353 30.6254,188.429 30.6268,188.507 30.6283,188.586 30.6298,188.665 30.6313,188.745 30.6329,188.824 30.6346,188.904 30.6363,188.983 
 30.638,189.062 30.6398,189.141 30.6417,189.221 30.6436,189.3 30.6455,189.381 30.6475,189.462 30.6495,189.543 30.6516,189.626 30.6537,189.709 30.6559,189.794 
 30.6581,189.879 30.6604,189.966 30.6627,190.052 30.665,190.14 30.6674,190.228 30.6699,190.315 30.6724,190.403 30.6749,190.491 30.6775,190.579 30.6801,190.666 
 30.6828,190.754 30.6856,190.841 30.6883,190.928 30.6912,191.015 30.694,191.103 30.697,191.191 30.6999,191.279 30.7029,191.368 30.706,191.457 30.7091,191.548 
 30.7123,191.638 30.7155,191.729 30.7187,191.821 30.722,191.913 30.7254,192.005 30.7287,192.097 30.7322,192.189 30.7357,192.28 30.7392,192.372 30.7428,192.463 
 30.7464,192.554 30.7501,192.645 30.7538,192.735 30.7576,192.826 30.7614,192.917 30.7652,193.008 30.7691,193.099 30.7731,193.19 30.7771,193.282 30.7811,193.375 
 30.7852,193.467 30.7894,193.56 30.7936,193.653 30.7978,193.747 30.8021,193.84 30.8064,193.933 30.8108,194.026 30.8152,194.119 30.8197,194.212 30.8242,194.304 
 30.8288,194.396 30.8334,194.488 30.8381,194.579 30.8428,194.67 30.8475,194.762 30.8523,194.853 30.8572,194.945 30.8621,195.036 30.867,195.128 30.872,195.221 
 30.877,195.313 30.8821,195.406 30.8873,195.499 30.8924,195.591 30.8977,195.684 30.9029,195.777 30.9083,195.869 30.9136,195.961 30.919,196.053 30.9245,196.144 
 30.93,196.235 30.9356,196.326 30.9412,196.416 30.9468,196.506 30.9525,196.596 30.9583,196.687 30.9641,196.777 30.9699,196.867 30.9758,196.957 30.9817,197.048 
 30.9877,197.138 30.9937,197.229 30.9998,197.32 31.0059,197.411 31.0121,197.501 31.0183,197.592 31.0245,197.682 31.0309,197.771 31.0372,197.861 31.0436,197.95 
 31.0501,198.038 31.0566,198.126 31.0631,198.214 31.0697,198.302 31.0763,198.39 31.083,198.477 31.0898,198.564 31.0965,198.652 31.1034,198.739 31.1102,198.827 
 31.1172,198.914 31.1241,199.002 31.1311,199.089 31.1382,199.177 31.1453,199.264 31.1525,199.351 31.1597,199.437 31.1669,199.524 31.1742,199.609 31.1816,199.695 
 31.1889,199.78 31.1964,199.864 31.2039,199.948 31.2114,200.032 31.219,200.116 31.2266,200.199 31.2343,200.283 31.242,200.366 31.2498,200.45 31.2576,200.533 
 31.2654,200.616 31.2733,200.7 31.2813,200.783 31.2893,200.866 31.2974,200.949 31.3055,201.031 31.3136,201.113 31.3218,201.195 31.33,201.276 31.3383,201.357 
 31.3466,201.437 31.355,201.517 31.3634,201.596 31.3719,201.675 31.3804,201.754 31.389,201.833 31.3976,201.911 31.4063,201.99 31.415,202.068 31.4237,202.146 
 31.4326,202.225 31.4414,202.303 31.4503,202.381 31.4592,202.458 31.4682,202.536 31.4773,202.613 31.4864,202.689 31.4955,202.766 31.5047,202.841 31.5139,202.917 
 31.5232,202.991 31.5325,203.065 31.5419,203.139 31.5513,203.213 31.5607,203.286 31.5702,203.359 31.5798,203.431 31.5894,203.504 31.599,203.576 31.6087,203.648 
 31.6185,203.72 31.6283,203.792 31.6381,203.864 31.648,203.936 31.6579,204.007 31.6679,204.078 31.6779,204.148 31.688,204.218 31.6981,204.287 31.7083,204.356 
 31.7185,204.424 31.7288,204.492 31.7391,204.559 31.7494,204.626 31.7598,204.692 31.7703,204.759 31.7808,204.825 31.7913,204.89 31.8019,204.956 31.8125,205.021 
 31.8232,205.087 31.8339,205.152 31.8447,205.217 31.855

In [5]:
#x_max = Int(ceil(maximum(kozai[:,1])))
y_s = zeros(time_max)
for i in 1:size(kozai,1)
    (x, y) = kozai[i,:]
    if ceil(Int, x) > time_max
        continue
    end
    y_s[ceil(Int, x)] = round(Int,y)
end
y_s -= mean(y_s)
dT = 1
yf = 2.0/(time_max/dT) * abs(rfft(y_s))
xf = linspace(0, 0.5/dT, div(time_max/dT,2))

pks= zeros(length(xf))
pks[findmax(yf[10:div(findmax(yf)[2],2)])[2]] = 0.1
1/xf[findmax(yf[10:div(findmax(yf)[2],2)])[2]]

6249.987499999999

In [6]:
plot_crop = 10000
plot(xf[1:min(plot_crop,end)], yf[1:min(plot_crop,end)])
plot!(xf[1:min(plot_crop,end)], pks[1:min(plot_crop,end)])

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0000 
 
 
 0.0025 
 
 
 0.0050 
 
 
 0.0075 
 
 
 0 
 
 
 5 
 
 
 10 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 21.7148,384.952 21.7722,384.636 21.8297,384.881 21.8871,384.756 21.9445,384.591 22.002,384.765 22.0594,384.804 22.1169,384.702 22.1743,384.638 22.2318,384.793 
 22.2892,384.644 22.3466,384.62 22.4041,384.725 22.4615,384.851 22.519,384.828 22.5764,384.809 22.6338,384.769 22.6913,384.726 22.7487,384.735 22.8062,384.707 
 22.8636,384.693 22.921,384.684 22.9785,384.808 23.0359,384.879 23.0934,384.775 23.1508,384.714 23.2082,384.715 23.2657,384.835 23.3231,384.726 23.3806,384.713 
 23.438,384.752 23.4954,384.804 23.5529,384.637 23.6103,384.73 23.6678,384.768 23.7252,384.565 23.7826,384.562 23.8401,384.693 23.8975,384.809 23.955,384.644 
 24.0124,384.706 24.0698,384.815 24.1273,384.705 24.1847,384.661 24.2422,384.857 24.2996,384.687 24.3571,384.646 24.4145,384.742 24.4719,384.741 24.5294,384.59 
 24.5868,384.693 24.6443,384.781 24.7017,384.813 24.7591,384.664 24.8166,384.898 24.874,384.737 24.9315,384.594 24.9889,384.47 25.0463,384.707 25.1038,384.798 
 25.1612,384.863 25.2187,384.733 25.2761,384.845 25.3335,384.661 25.391,384.857 25.4484,384.643 25.5059,384.657 25.5633,384.501 25.6207,384.665 25.6782,384.702 
 25.7356,384.756 25.7931,384.804 25.8505,384.853 25.9079,384.724 25.9654,384.7 26.0228,384.697 26.0803,384.692 26.1377,384.596 26.1951,384.648 26.2526,384.789 
 26.31,384.846 26.3675,384.706 26.4249,384.759 26.4824,384.852 26.5398,384.784 26.5972,384.597 26.6547,384.579 26.7121,384.849 26.7696,384.694 26.827,384.714 
 26.8844,384.678 26.9419,384.78 26.9993,384.559 27.0568,384.622 27.1142,384.802 27.1716,384.672 27.2291,384.706 27.2865,384.72 27.344,384.724 27.4014,384.466 
 27.4588,384.775 27.5163,384.762 27.5737,384.579 27.6312,384.302 27.6886,384.553 27.746,384.782 27.8035,384.846 27.8609,384.838 27.9184,384.818 27.9758,384.807 
 28.0332,384.669 28.0907,384.767 28.1481,384.689 28.2056,384.746 28.263,384.637 28.3205,384.644 28.3779,384.919 28.4353,384.738 28.4928,384.439 28.5502,384.526 
 28.6077,384.855 28.6651,384.578 28.7225,384.295 28.78,384.568 28.8374,384.479 28.8949,384.863 28.9523,384.603 29.0097,384.81 29.0672,384.367 29.1246,384.818 
 29.1821,384.462 29.2395,384.38 29.2969,384.084 29.3544,384.607 29.4118,384.622 29.4693,384.578 29.5267,384.526 29.5841,384.814 29.6416,384.452 29.699,384.501 
 29.7565,384.61 29.8139,384.643 29.8713,384.198 29.9288,384.396 29.9862,384.241 30.0437,384.536 30.1011,384.492 30.1585,384.761 30.216,384.477 30.2734,384.547 
 30.3309,384.065 30.3883,384.053 30.4458,383.861 30.5032,384.6 30.5606,384.7 30.6181,384.623 30.6755,384.302 30.733,384.698 30.7904,384.215 30.8478,384.824 
 30.9053,384.288 30.9627,384.239 31.0202,383.289 31.0776,384.091 31.135,382.001 31.1925,381.697 31.2499,381.732 31.3074,378.722 31.3648,378.768 31.4222,377.816 
 31.4797,380.888 31.5371,382.375 31.5946,383.637 31.652,383.699 31.7094,384.003 31.7669,384.108 31.8243,383.938 31.8818,383.767 31.9392,384.142 31.9966,384.636 
 32.0541,384.34 32.1115,384.459 32.169,384.83 32.2264,384.279 32.2838,384.407 32.3413,384.22 32.3987,384.602 32.4562,384.458 32.5136,384.566 32.5711,384.561 
 32.6285,384.53 32.6859,384.728 32.7434,384.831 32.8008,384.739 32.8583,384.635 32.9157,384.409 32.9731,384.671 33.0306,384.714 33.088,384.747 33.1455,384.584 
 33.2029,384.693 33.2603,384.623 33.3178,384.479 33.3752,384.648 33.4327,384.742 33.4901,384.605 33.5475,384.31 33.605,384.376 33.6624,384.56 33.7199,384.645 
 33.7773,384.819 33.8347,384.838 33.8922,384.718 33.9496,384.858 34.0071,384.789 34.0645,384.535 34.1219,384.313 34.1794,384.837 34.2368,384.235 34.2943,384.883 
 34.3517,384.294 34.4092,384.139 34.4666,384.574 34.524,384.546 34.5815,384.564 34.6389,384.454 34.6964,384.528 34.7538,384.749 34.8112,384.594 34.8687,384.417 
 34.9261,384.71 34.98

In [7]:
anim_dt=1
@gif for i in 1:anim_dt:500
    @printf("%d\r", i)
    flush(STDOUT)
    #plot(map(x -> x[1], prey[1:i]), map(x -> x[2], prey[1:i]))
    plot(kozai[1:i,1], kozai[1:i,2])
end every 1

INFO: Saved animation to /home/pythonnut/Documents/prog/sipz/tmp.gif


Plots.AnimatedGif("/home/pythonnut/Documents/prog/sipz/tmp.gif")

In [8]:
X = 40:1:400
periods = mass_to_period(X);

#for x in zip(X, periods)
#    @printf("%d,%f\n", x[1], x[2])
#end

Allocating memory... done.


In [9]:
plot(X, periods)
fit = polyfit(X, periods,2)
plot!(X, map(model(fit), X))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 100 
 
 
 200 
 
 
 300 
 
 
 400 
 
 
 10000 
 
 
 20000 
 
 
 30000 
 
 
 40000 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 48.3815,383.387 49.9028,383.258 51.4241,383.158 52.9455,382.604 54.4668,381.112 55.9881,381.288 57.5095,381.063 59.0308,381.16 60.5522,381.757 62.0735,378.878 
 63.5948,378.455 65.1162,378.716 66.6375,378.946 68.1588,379.014 69.6802,378.991 71.2015,379.256 72.7229,374.638 74.2442,377.445 75.7655,377.849 77.2869,377.333 
 78.8082,375.483 80.3295,374.483 81.8509,376.349 83.3722,375.553 84.8936,372.681 86.4149,373.407 87.9362,375.306 89.4576,371 90.9789,371.485 92.5002,371.326 
 94.0216,371.218 95.5429,365.996 97.0643,369.961 98.5856,372 100.107,371.164 101.628,370.833 103.15,369.276 104.671,367.85 106.192,368.064 107.714,368.205 
 109.235,365.661 110.756,364.784 112.278,362.11 113.799,365.229 115.32,365.053 116.842,366.243 118.363,360.987 119.884,362.432 121.406,360.87 122.927,362.432 
 124.448,360.987 125.97,359.27 127.491,360.514 129.012,359.904 130.534,356.322 132.055,360.028 133.576,356.322 135.098,356.474 136.619,360.028 138.14,356.168 
 139.662,355.542 141.183,354.053 142.704,353.706 144.226,353.53 145.747,352.441 147.268,350.485 148.79,350.485 150.311,350.891 151.832,350.485 153.354,348.333 
 154.875,351.29 156.396,348.558 157.918,347.409 159.439,347.644 160.96,343.58 162.482,347.409 164.003,344.127 165.524,344.127 167.046,344.396 168.567,344.127 
 170.088,340.649 171.61,342.157 173.131,340.337 174.652,340.337 176.174,342.157 177.695,339.05 179.216,339.05 180.738,338.043 182.259,336.997 183.78,337.698 
 185.302,336.64 186.823,334.396 188.345,337.35 189.866,331.542 191.387,329.783 192.909,332.798 194.43,330.675 195.951,331.111 197.473,329.783 198.994,329.783 
 200.515,326.45 202.037,325.945 203.558,325.945 205.079,325.945 206.601,326.947 208.122,325.945 209.643,322.746 211.165,322.746 212.686,322.184 214.207,319.228 
 215.729,316.681 217.25,317.334 218.771,317.976 220.293,318.608 221.814,315.34 223.335,316.681 224.857,318.608 226.378,313.951 227.899,313.951 229.421,313.951 
 230.942,312.512 232.463,311.019 233.985,309.471 235.506,308.674 237.027,309.471 238.549,306.191 240.07,306.191 241.591,306.191 243.113,305.33 244.634,306.191 
 246.155,299.786 247.677,300.758 249.198,301.71 250.719,298.793 252.241,301.71 253.762,295.684 255.283,297.779 256.805,297.779 258.326,296.743 259.847,293.495 
 261.369,296.743 262.89,294.602 264.411,292.363 265.933,288.807 267.454,292.363 268.975,288.807 270.497,286.294 272.018,287.565 273.539,287.565 275.061,284.991 
 276.582,284.991 278.103,283.656 279.625,282.288 281.146,282.288 282.667,279.447 284.189,282.288 285.71,277.971 287.231,277.971 288.753,277.971 290.274,277.971 
 291.795,274.9 293.317,271.662 294.838,273.303 296.359,271.662 297.881,269.975 299.402,266.457 300.923,268.241 302.445,268.241 303.966,268.241 305.488,269.975 
 307.009,266.457 308.53,264.621 310.052,262.732 311.573,262.732 313.094,260.786 314.616,260.786 316.137,256.714 317.658,256.714 319.18,254.583 320.701,256.714 
 322.222,250.115 323.744,252.385 325.265,252.385 326.786,250.115 328.308,250.115 329.829,250.115 331.35,247.771 332.872,247.771 334.393,245.348 335.914,242.844 
 337.436,245.348 338.957,240.253 340.478,245.348 342,240.253 343.521,240.253 345.042,234.794 346.564,234.794 348.085,237.571 349.606,234.794 351.128,234.794 
 352.649,228.931 354.17,231.916 355.692,228.931 357.213,228.931 358.734,228.931 360.256,228.931 361.777,222.616 363.298,225.833 364.82,225.833 366.341,222.616 
 367.862,219.273 369.384,215.796 370.905,219.273 372.426,215.796 373.948,219.273 375.469,215.796 376.99,215.796 378.512,212.178 380.033,212.178 381.554,212.178 
 383.076,212.178 384.597,208.408 386.118,204.478 387.64,204.478 389.161,204.478 390.682,204.478 392.204,200.378 393.725,200.378 395.246,200.378 396.768,196.095 
 398.289,196

In [10]:
plot(X, periods- map(model(fit), X))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 100 
 
 
 200 
 
 
 300 
 
 
 400 
 
 
 -1000 
 
 
 -500 
 
 
 0 
 
 
 500 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 48.3815,147.835 49.9028,153.503 51.4241,159.895 52.9455,156.616 54.4668,133.286 55.9881,145.826 57.5095,149.848 59.0308,160.862 60.5522,182.665 62.0735,129.991 
 63.5948,130.072 65.1162,144.927 66.6375,159.181 68.1588,170.031 69.6802,179.03 71.2015,194.268 72.7229,104.854 74.2442,174.775 75.7655,193.248 77.2869,192.059 
 78.8082,162.323 80.3295,150.912 81.8509,201.058 83.3722,194.167 84.8936,142.833 86.4149,168.768 87.9362,219.936 89.4576,138.072 90.9789,159.075 92.5002,166.327 
 94.0216,174.771 95.5429,73.5906 97.0643,169.538 98.5856,224.272 100.107,217.407 101.628,221.459 103.15,199.289 104.671,180.009 106.192,195.987 107.714,210.472 
 109.235,167.437 110.756,160.251 112.278,114.593 113.799,193.292 115.32,201.37 116.842,238.845 118.363,138.111 119.884,181.208 121.406,159.888 122.927,205.649 
 124.448,186.993 125.97,162.592 127.491,201.777 129.012,201.261 130.534,137.074 132.055,229.313 133.576,162.627 135.098,178.783 136.619,268 138.14,198.263 
 139.662,197.966 141.183,179.229 142.704,185.075 144.226,194.673 145.747,184.757 147.268,156.346 148.79,169.956 150.311,192.354 151.832,197.415 153.354,165.085 
 154.875,242.443 156.396,197.847 157.918,187.303 159.439,206.496 160.96,133.565 162.482,230.04 164.003,174.035 165.524,188.519 167.046,208.855 168.567,217.725 
 170.088,157.839 171.61,204.978 173.131,180.823 174.652,195.784 176.174,249.859 177.695,198.332 179.216,213.531 180.738,207.201 182.259,200.129 183.78,230.612 
 185.302,223.416 186.823,190.879 188.345,269.924 189.866,161.084 191.387,139.183 192.909,219.784 194.43,190.223 195.951,215.661 197.473,203.314 198.994,219.545 
 200.515,164.359 202.037,169.916 203.558,186.385 205.079,202.934 206.601,241.066 208.122,236.27 209.643,184.449 211.165,201.315 212.686,206.186 214.207,159.818 
 215.729,122.273 217.25,153.47 218.771,184.508 220.293,215.395 221.814,162.723 223.335,208.988 224.857,267.901 226.378,185.676 227.899,203.416 229.421,221.236 
 230.942,208.261 232.463,194.222 233.985,179.055 235.506,180.105 237.027,215.409 238.549,163.345 240.07,181.721 241.591,200.176 243.113,200.249 244.634,237.324 
 246.155,118.631 247.677,158.257 249.198,197.532 250.719,153.89 252.241,235.474 253.762,125.292 255.283,189.406 256.805,208.655 258.326,205.755 259.847,155.483 
 261.369,244.651 262.89,218.28 264.411,189.9 265.933,133.35 267.454,229.431 268.975,173.039 270.497,139.089 272.018,186.41 273.539,206.533 275.061,171.514 
 276.582,191.796 278.103,183.523 279.625,174.615 281.146,195.135 282.667,154.778 284.189,236.412 285.71,164.55 287.231,185.387 288.753,206.304 290.274,227.3 
 291.795,182.512 293.317,134.195 294.838,190.635 296.359,176.743 297.881,161.953 299.402,107.957 300.923,167.775 302.445,189.407 303.966,211.118 305.488,270.111 
 307.009,216.512 308.53,199.087 310.052,180.582 311.573,202.691 313.094,183.135 314.616,205.402 316.137,140.409 317.658,162.835 319.18,139.624 320.701,207.925 
 322.222,89.0149 323.744,160.448 325.265,183.271 326.786,157.484 328.308,180.466 329.829,203.527 331.35,176.381 332.872,199.602 334.393,170.939 335.914,140.594 
 337.436,217.776 338.957,132.014 340.478,264.931 342,179.328 343.521,203.104 345.042,109.852 346.564,133.787 348.085,217.382 349.606,181.895 351.128,206.068 
 352.649,104.539 354.17,192.906 355.692,153.283 357.213,177.773 358.734,202.344 360.256,226.993 361.777,116.265 363.298,210.08 364.82,234.968 366.341,190.929 
 367.862,144.263 369.384,94.8085 370.905,194.595 372.426,145.299 373.948,245.244 375.469,196.107 376.99,221.63 378.512,169.608 380.033,195.29 381.554,221.052 
 383.076,246.892 384.597,191.953 386.118,133.653 387.64,159.732 389.161,185.891 390.682,212.129 392.204,150.48 393.725,176.877 395.246,203.354 396.768,138.034 
 398.289,164.6

In [11]:
fit

3-element Array{Any,1}:
 -514.374   
   27.5079  
    0.208799